# Préparation des données pour l’API + Packaging MLflow 

## Objectif

Cette étape prépare un dataset exploitable en production pour alimenter l’API de scoring (Gradio) et la base SQLite.

L’objectif est de :
- construire un fichier `X_api.csv` contenant uniquement les features attendues par le modèle final,
- garantir la cohérence entre les données utilisées en entraînement (Partie 1) et les données servies en production (Partie 2),
- centraliser les artefacts nécessaires dans MLflow pour assurer la traçabilité.

## Chargement des données de production simulées

Nous utilisons `test_final.csv` (dataset Kaggle sans variable TARGET) comme proxy de données réelles en production.

Ce fichier représente des demandes clients pour lesquelles on souhaite produire un score de défaut via l’API.

In [ ]:
from pathlib import Path
import pandas as pd

CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

df = pd.read_csv(DATA_PROCESSED / "test_final.csv")


## Création du dataset API-ready

Nous construisons un dataset `X_api.csv` contenant :
- `SK_ID_CURR` (identifiant client),
- les 125 features retenues.

Ce fichier sera ensuite utilisé pour alimenter une base SQLite locale afin de simuler le stockage des données en production.

La colonne **SK_ID_CURR** est indispensable pour pouvoir interroger un client spécifique via l’API.

In [ ]:

FEATURE_REDUCTION_DIR = PROJECT_ROOT / "reports" / "feature_reduction"
FEATURE_SET_NAME = "top125_nocorr"
kept_file = FEATURE_REDUCTION_DIR / f"kept_features_{FEATURE_SET_NAME}.txt"

kept_features = [
    l.strip()
    for l in kept_file.read_text(encoding="utf-8").splitlines()
    if l.strip()
]
kept_features = [c for c in kept_features if c in df.columns]
if len(kept_features) == 0:
    raise ValueError(f"kept_features vide -> vérifie le contenu de: {kept_file}")


cols = ["SK_ID_CURR"] + kept_features
missing = [c for c in cols if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans test_final.csv: {missing[:10]}")

## Export du fichier `X_api.csv`

Le fichier est exporté dans `data/processed/`.

Il constitue une source standardisée pour :
- initialiser la base SQLite,
- simuler des appels API,
- générer des données de monitoring (logs de prédiction, scores, latence, drift).

In [ ]:


X_api = df[cols].copy()

out_path = DATA_PROCESSED / "X_api.csv"
X_api.to_csv(out_path, index=False)

print("OK:", out_path, X_api.shape)

OK: c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\data\processed\X_api.csv (48744, 126)


## Récupération de la liste des features du modèle final (Top 125)

Le modèle final (CatBoost) a été entraîné sur un jeu de variables réduit afin de :
- diminuer le temps d’entraînement,
- améliorer la stabilité du modèle,
- faciliter l’interprétabilité.

La liste des variables retenues est stockée dans :
`reports/feature_reduction/kept_features_top125_nocorr.txt`

Nous relisons ce fichier pour garantir que l’API utilisera exactement les mêmes colonnes que le modèle final.

## Packaging MLflow : traçabilité des features et des données API

Dans une démarche MLOps, il est important de conserver les artefacts critiques liés au modèle.

Nous créons donc un nouveau run MLflow dédié au packaging afin de logger :
- la liste des features retenues (`kept_features_...txt`),
- le dataset API (`X_api.csv` ou un sample),
- les paramètres associés (nom du feature set, nombre de variables).

Cela garantit la reproductibilité et la traçabilité de la pipeline de mise en production.

In [4]:
import mlflow
from mlflow.tracking import MlflowClient


MLFLOW_DB = PROJECT_ROOT / "mlflow.db"
mlflow.set_tracking_uri("sqlite:///" + MLFLOW_DB.as_posix())

client = MlflowClient()
print("Tracking URI:", mlflow.get_tracking_uri())

MODEL_NAME = "home_credit_catboost"
SOURCE_MODEL_URI = f"models:/{MODEL_NAME}/2"   
EXPERIMENT_NAME = "home_credit_final_benchmark"
mlflow.set_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name="package_api_v3") as run:
    run_id = run.info.run_id

    mlflow.log_param("feature_set_name", FEATURE_SET_NAME)
    mlflow.log_param("n_features", len(kept_features))

    # Artefact: liste des features
    mlflow.log_artifact(str(kept_file), artifact_path="feature_selection")
    # Artefact : x_api
    mlflow.log_artifact(str(out_path), artifact_path="api_data")

    print("Packaging run:", run_id)

Tracking URI: sqlite:///c:/Users/yoann/Documents/open classrooms/projet 8/livrables/pret a dépenser/mlflow.db
Packaging run: edfe64f1e42d4da385d599f51672ce48


## Packaging MLflow : traçabilité des features et des données API

Dans une démarche MLOps, il est important de conserver les artefacts critiques liés au modèle.

Nous créons donc un nouveau run MLflow dédié au packaging afin de logger :
- la liste des features retenues (`kept_features_...txt`),
- le dataset API (`X_api.csv` ou un sample),
- les paramètres associés (nom du feature set, nombre de variables).

Cela garantit la reproductibilité et la traçabilité de la pipeline de mise en production.

In [3]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# 1) récupérer le run_id + source de V2
mv2 = client.get_model_version(name=MODEL_NAME, version="2")
print("V2 source:", mv2.source)

# 2) enregistrer une nouvelle version à partir de la source de V2

registered = mlflow.register_model(model_uri=mv2.source, name=MODEL_NAME)

print("Registered new version:", registered.version)

# 3) tagger la V3 avec le run packaging + feature set
new_v = registered.version
client.set_model_version_tag(MODEL_NAME, new_v, "feature_set_name", FEATURE_SET_NAME)
client.set_model_version_tag(MODEL_NAME, new_v, "n_features", str(len(kept_features)))
client.set_model_version_tag(MODEL_NAME, new_v, "packaging_run_id", run_id)

client.update_model_version(
    name=MODEL_NAME,
    version=new_v,
    description=f"V{new_v}: packaging + artefacts API (kept_features, X_api). Packaging run: {run_id}"
)

V2 source: file:///C:/Users/yoann/Documents/open%20classrooms/projet%208/livrables/pret%20a%20d%C3%A9penser/artifacts/d7def84a96344b079343de1bf700981b/artifacts/model
Registered new version: 3


Registered model 'home_credit_catboost' already exists. Creating a new version of this model...
Created version '3' of model 'home_credit_catboost'.


<ModelVersion: aliases=[], creation_timestamp=1771334017685, current_stage='None', description=('V3: packaging + artefacts API (kept_features, X_api). Packaging run: '
 'a63d5498eaa04b10bd1ed36d65583300'), last_updated_timestamp=1771334017724, name='home_credit_catboost', run_id=None, run_link=None, source='file:///C:/Users/yoann/Documents/open%20classrooms/projet%208/livrables/pret%20a%20d%C3%A9penser/artifacts/d7def84a96344b079343de1bf700981b/artifacts/model', status='READY', status_message=None, tags={'feature_set_name': 'top125_nocorr',
 'n_features': '125',
 'packaging_run_id': 'a63d5498eaa04b10bd1ed36d65583300'}, user_id=None, version=3>

## Résultat attendu

À la fin de ce notebook, nous disposons de :

- `data/processed/X_api.csv` : dataset prêt pour l’API
- un run MLflow "packaging" contenant les artefacts clés
- une nouvelle version du modèle dans le Registry (V3) utilisable pour le déploiement

Ce modèle sera ensuite chargé par l’API Gradio afin de fournir des prédictions en quasi temps réel.